In [28]:
import folium
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import numpy as np

In [2]:
kol_lat = 22.5726
kol_lon = 88.3639
print('The coordinates of Kolkata, India is {} N, {} E'.format(kol_lat, kol_lon))

The coordinates of Kolkata, India is 22.5726 N, 88.3639 E


In [4]:
kol_map = folium.Map(location=[kol_lat, kol_lon], zoom_start=13)

In [5]:
bang_lat = 12.9716
bang_lon = 77.5946
print('The coordinates of Bangaluru, India is {} N, {} E'.format(bang_lat, bang_lon))

The coordinates of Bangaluru, India is 12.9716 N, 77.5946 E


In [6]:
bang_map = folium.Map(location=[bang_lat, bang_lon], zoom_start=12)

In [7]:
Creds = pd.read_csv('FourSquareCredentials.csv')
CLIENT_ID = Creds['CLIENT_ID'][0] # your Foursquare ID
CLIENT_SECRET = Creds['CLIENT_SECRET'][0] # your Foursquare Secret
VERSION = Creds['VERSION'][0] # Foursquare API version

In [8]:
radius = 6000
LIMIT = 200
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    kol_lat, 
    kol_lon, 
    radius, 
    LIMIT)

In [9]:
results = requests.get(url).json()

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
def get_venue_rating(VENUE_ID):
    url2 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
        VENUE_ID,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        kol_lat, 
        kol_lon, 
        radius, 
        LIMIT)
    results2 = requests.get(url2).json()
    return VENUE_ID, results2['response']['venue']['rating']

In [12]:
venues = results['response']['groups'][0]['items']
    
kol_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.id', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
kol_venues =kol_venues.loc[:, filtered_columns]

# filter the category for each row
kol_venues['venue.categories'] = kol_venues.apply(get_category_type, axis=1)
# clean columns
kol_venues.columns = [col.split(".")[-1] for col in kol_venues.columns]

kol_venues.shape

(100, 5)

In [13]:
for lat, lng, cat, name in zip(kol_venues['lat'], kol_venues['lng'], kol_venues['categories'], kol_venues['name']):
    label = '{} ({})'.format(name, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        fill_color='##3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(kol_map)  
    
kol_map

In [14]:
kol_venues.sample(10)

,id,name,categories,lat,lng
77,4d5f52e3ef378cfa06736ea6,Yuvabharati Krirangan (Salt Lake Stadium),Soccer Stadium,22.569197,88.408232
78,4f1933b0e4b0a339fac6e75e,Avani Riverside Mall,Shopping Mall,22.563241,88.323725
54,4d3879193ffba14346df5c56,The Hub,Italian Restaurant,22.537879,88.334498
63,4ce160d9df986ea8c649ec16,Stadel,Diner,22.569257,88.411274
68,4d0111e6f7988eecee2b5de7,PVR (XL),Multiplex,22.577859,88.400579
9,4d201719d7b0b1f76417129f,Oxford Bookstore,Bookstore,22.553652,88.351732
25,4f179ce1e4b035bfa73f9f27,Princep Ghat,Harbor / Marina,22.559267,88.333057
82,54f84dce498e211ce5c20c9e,Carnival Cinemas,Movie Theater,22.573300,88.413693
47,4c9e20b754c8a1cd465f8a4b,New Sharma's Dhaba,Dhaba,22.534093,88.356786
14,4c0218258ef2c9b66d9c16fc,Victoria Memorial,History Museum,22.545844,88.342890


In [21]:
kol_venues.drop('id', axis = 1, inplace = True)

In [23]:
kol_venues.sample(10)

,name,categories,lat,lng
4,Peter Cat,Indian Restaurant,22.552365,88.352544
91,Cafe Coffee Day,Café,22.606198,88.402900
31,Spaghetti Kitchen,Italian Restaurant,22.538601,88.351458
94,Cafe Coffee Day,Café,22.573182,88.412693
46,Hotel Hindusthan International,Hotel,22.541075,88.350490
43,6 Ballygunge Place,Bengali Restaurant,22.527712,88.368677
60,Mani Square Mall,Shopping Mall,22.577823,88.400591
51,Sonargaon,Indian Restaurant,22.537545,88.334210
59,The Cream & Fudge Factory,Bakery,22.534549,88.364854
95,PVR Cinemas,Multiplex,22.563216,88.323704


In [25]:
zomato_key = pd.DataFrame(dict({'user-key': ['6519e1585a8a39bbad1ff1217e707d63']}))
zomato_key.to_csv('zomato_key.csv')

In [32]:
zomato_key = pd.read_csv('zomato_key.csv')
headers = {'user-key': zomato_key['user-key'][0]}
venues_information = []

for _, row in kol_venues.iterrows():
    venue = []
    url = ('https://developers.zomato.com/api/v2.1/search?q={}' + 
          '&start=0&count=1&lat={}&lon={}&sort=real_distance').format(row['name'], row['lat'], row['lng'])
    result = requests.get(url, headers = headers).json()
    if (len(result['restaurants']) > 0):
        venue.append(result['restaurants'][0]['restaurant']['name'])
        venue.append(result['restaurants'][0]['restaurant']['location']['latitude'])
        venue.append(result['restaurants'][0]['restaurant']['location']['longitude'])
        venue.append(result['restaurants'][0]['restaurant']['average_cost_for_two'])
        venue.append(result['restaurants'][0]['restaurant']['price_range'])
        venue.append(result['restaurants'][0]['restaurant']['user_rating']['aggregate_rating'])
        venue.append(result['restaurants'][0]['restaurant']['location']['address'])
        venues_information.append(venue)
    else:
        print(row)

zomato_kol_venues = pd.DataFrame(venues_information, 
                                  columns = ['venue', 'latitude', 
                                             'longitude', 'price_for_two', 
                                             'price_range', 'rating', 'address'])

name          Crossword Bookstores Ltd
categories                   Bookstore
lat                            22.5876
lng                            88.4084
Name: 61, dtype: object


In [31]:
zomato_kol_venues

,venue,latitude,longitude,price_for_two,price_range,rating,address
0,Baan Thai - The Oberoi Grand,22.5619715012,88.3519555628,4000,4,4.2,"The Oberoi Grand, 15, Jawaharlal Nehru Road, N..."
1,Shree Balaji South Indian Mess Veg,22.5701181810,88.3265332505,200,1,3.5,"Shop 11, Block A2, 106 Kiran Chandra Singha Ro..."
2,The Legacy Grill - The Lalit Great Eastern,22.5678432397,88.3500756696,4000,4,3.9,"The Lalit Great Eastern, 1, 2, 3, Old Court Ho..."
3,Blue And Beyond,22.5591347721,88.3531652391,1600,3,3.9,"8A & 8B, The Lindsay, Lindsay Street, New Mark..."
4,Carpe Diem,22.5522216648,88.3525932580,1000,3,4.5,"18M Park Street Area, Kolkata"
5,Blue Poppy-Thakali,22.5486174352,88.3512481302,500,2,4.3,"Sikkim Commercial House, 1st Floor, 4/1 Middle..."
6,Aqua - The Park,22.5538351774,88.3518341929,3000,4,4.1,"The Park, 17, Park Street Area, Park Street ..."
7,Cafe Coffee Day,22.5775520000,88.3903200000,600,2,3.6,"P-183 A, CIT Road, Scheme 6M, Kankurgachhi, Ko..."
8,Chittaranjan Mistanna Bhandar,22.5973876166,88.3657558635,100,1,4.3,"34B, Shobha Bazar Street, Near Lal Mandir, Sho..."
9,Cha Bar,22.5537528147,88.3518875018,300,1,4.1,"Oxford Book Store, 17, Park Street Area, Kolka..."


In [34]:
final_df = zomato_kol_venues.drop('address', axis=1)

In [35]:
final_df.sample(10)

,venue,latitude,longitude,price_for_two,price_range,rating
65,Cafe Coffee Day,22.5946570000,88.4084360000,600,2,3.6
34,Bombay Brasserie,22.5387375236,88.3656009659,1900,3,4.2
89,Pizza Hut,22.5626117852,88.3231053874,500,2,3.7
42,The French Loaf,22.5386015794,88.3543772623,300,1,3.8
78,Afraa Restaurant And Lounge,22.5877901116,88.4081216902,1800,3,4.2
6,Aqua - The Park,22.5538351774,88.3518341929,3000,4,4.1
31,WOW! Momo,22.5385018661,88.3513698354,400,1,3.8
47,Sharma Dhaba,22.5338725547,88.3568103611,700,2,3.8
37,Starbucks Coffee,22.5525275865,88.3529057354,700,2,4.2
96,Domino's Pizza,22.6210375600,88.3491915500,400,1,3.6
